In [6]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
import time
import trajectory_msgs.msg
import tf.transformations as tf
from math import pi
from std_msgs.msg import String,Empty,UInt16
from moveit_commander.conversions import pose_to_list

import helper

In [2]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('dual_arm_origami', anonymous=True)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
rospy.sleep(2)
group_name1 = "hong_arm"
group1 = moveit_commander.MoveGroupCommander(group_name1)
group_name2 = "kong_arm"
group2 = moveit_commander.MoveGroupCommander(group_name2)
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

In [3]:
def addCollisionObjects():
    box_pose1 = geometry_msgs.msg.PoseStamped()
    box_pose1.header.frame_id = "world"
    box_pose1.pose.orientation.w = 1.0
    box_pose1.pose.position.x = -0.105
    box_pose1.pose.position.y = -0.1485
    box_pose1.pose.position.z = 0.7055
    box_name = 'box1'
    scene.add_box(box_name, box_pose1, (0.01, 0.01, 0.01))
    rospy.sleep(1)
    box_pose2 = geometry_msgs.msg.PoseStamped()
    box_pose2.header.frame_id = "world"
    box_pose2.pose.orientation.w = 1.0
    box_pose2.pose.position.x = 0.105
    box_pose2.pose.position.y = -0.1485
    box_pose2.pose.position.z = 0.7055
    box_name = 'box2'
    scene.add_box(box_name, box_pose2, (0.01, 0.01, 0.01))
    rospy.sleep(1)
    box_pose3 = geometry_msgs.msg.PoseStamped()
    box_pose3.header.frame_id = "world"
    box_pose3.pose.orientation.w = 1.0
    box_pose3.pose.position.x = -0.105
    box_pose3.pose.position.y = 0.1485
    box_pose3.pose.position.z = 0.7055
    box_name = 'box3'
    scene.add_box(box_name, box_pose3, (0.01, 0.01, 0.01))
    
    box_pose4 = geometry_msgs.msg.PoseStamped()
    box_pose4.header.frame_id = "world"
    box_pose4.pose.orientation.w = 1.0
    box_pose4.pose.position.x = -0.105
    box_pose4.pose.position.y = 0.1485
    box_pose4.pose.position.z = 0.7055
    box_name = 'box4'
    scene.add_box(box_name, box_pose4, (0.01, 0.01, 0.01))

In [4]:
def robkong_go_to_home():
    joint_values = [-1.64, -0.73, 1.2, -1.94, -1.64, 0.73]
    group2.set_joint_value_target(joint_values)
    group2.go(joint_values, wait=True)
    group2.stop()
    group2.clear_pose_targets()

def robhong_go_to_home():
    joint_values = [-1.41, -1.49, 2.2, -2.28, -1.57, 0.16]
    group1.set_joint_value_target(joint_values)
    group1.go(joint_values, wait=True)
    group1.stop()
    group1.clear_pose_targets()

In [5]:
def move_waypoints(positions,robot_arm):
    waypoints = []
    waypoints.append(group1.get_current_pose().pose)
    wpose = copy.deepcopy(group1.get_current_pose().pose)
    wpose.position.x += positions.x
    wpose.position.y += positions.y
    wpose.position.z += positions.z
    waypoints.append(copy.deepcopy(wpose))
    (plan, fraction) = group1.compute_cartesian_path(
                                        waypoints,   # waypoints to follow
                                        0.01,        # eef_step
                                        0.0)         # jump_threshold
    if int(robot_arm) == 1:
        group1.go(plan)
    elif int(robot_arm) == 2:
        group2.go(plan)

def scale_trajectory_speed(traj, scale):
    new_traj = RobotTrajectory()
    new_traj.joint_trajectory = traj.joint_trajectory
    n_joints = len(traj.joint_trajectory.joint_names)
    n_points = len(traj.joint_trajectory.points)
    points = list(traj.joint_trajectory.points)

    for i in range(n_points):
        point = JointTrajectoryPoint()
        point.time_from_start = traj.joint_trajectory.points[i].time_from_start / scale
        point.velocities = list(traj.joint_trajectory.points[i].velocities)
        point.accelerations = list(traj.joint_trajectory.points[i].accelerations)
        point.positions = traj.joint_trajectory.points[i].positions

        for j in range(n_joints):
            point.velocities[j] = point.velocities[j] * scale
            point.accelerations[j] = point.accelerations[j] * scale * scale
        points[i] = point

    new_traj.joint_trajectory.points = points
    return new_traj

In [ ]:
def tilt(point, axis, angle, velocity):
    '''Tilt primitive motion of robot.
    Parameters:
        point (list): 3-D coordinate of point in rotation axis
        axis (list): 3-D vector of rotation axis (right-hand rule)
        angle (double): angle of tilting
        velocity (double): robot velocity between 0 and 1
    Returns:

    '''
    # Normalize axis vector
    axis = axis / numpy.linalg.norm(axis)

    # Pose variables. The parameters can be seen from "$ rosmsg show Pose"
    pose_target = group.get_current_pose().pose
    pos_initial = [pose_target.position.x, pose_target.position.y, pose_target.position.z]
    ori_initial = [pose_target.orientation.x, pose_target.orientation.y, pose_target.orientation.z, pose_target.orientation.w]

    # Tilt center point. Closest point from tcp to axis line
    center = numpy.add(point, numpy.dot(numpy.subtract(pos_initial, point), axis)*axis)

    # Closest distance from tcp to axis line
    radius = numpy.linalg.norm(numpy.subtract(center, pos_initial))

    # Pair of orthogonal vectors in tilt plane
    v1 =  -numpy.subtract(numpy.add(center, numpy.dot(numpy.subtract(pos_initial, center), axis)*axis), pos_initial)
    v1 = v1/numpy.linalg.norm(v1)
    v2 = numpy.cross(axis, v1)

    # Interpolate orientation poses via quaternion slerp
    q = helper.axis_angle2quaternion(axis, angle)
    ori_target = tf.transformations.quaternion_multiply(q, ori_initial)
    ori_waypoints = helper.slerp(ori_initial, ori_target, numpy.arange(1.0/angle , 1.0+1.0/angle, 1.0/angle))

    waypoints = []
    for t in range(1, angle + 1):
        circle = numpy.add(center, radius*(math.cos(math.radians(t)))*v1 + radius*(math.sin(math.radians(t)))*v2)
        pose_target.position.x = circle[0]
        pose_target.position.y = circle[1]
        pose_target.position.z = circle[2]
        pose_target.orientation.x = ori_waypoints[t-1][0]
        pose_target.orientation.y = ori_waypoints[t-1][1]
        pose_target.orientation.z = ori_waypoints[t-1][2]
        pose_target.orientation.w = ori_waypoints[t-1][3]
        waypoints.append(copy.deepcopy(pose_target))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0) # waypoints, resolution=1cm, jump_threshold)
    rospy.sleep(1)
    retimed_plan = group.retime_trajectory(robot.get_current_state(), plan, velocity) # Retime trajectory with scaled velocity
    rospy.sleep(2)
    group.execute(retimed_plan)
    rospy.sleep(2)